In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

mug_csv = pd.read_csv('mug.csv')
kettle_csv = pd.read_csv('kettle.csv')
bottle_csv = pd.read_csv('bottle.csv')
bakingtray_csv = pd.read_csv('bakingtray.csv')
hammer_csv = pd.read_csv('hammer.csv')
pan_csv = pd.read_csv('pan.csv')
wrench_csv = pd.read_csv('wrench.csv')


mug_csv['Object'] = ['mug'] * len(mug_csv)
kettle_csv['Object'] = ['kettle'] * len(kettle_csv)
bottle_csv['Object'] = ['bottle'] * len(bottle_csv)
bakingtray_csv['Object'] = ['bakingtray'] * len(bakingtray_csv)
hammer_csv['Object'] = ['hammer'] * len(hammer_csv)
pan_csv['Object'] = ['pan'] * len(pan_csv)
wrench_csv['Object'] = ['wrench'] * len(wrench_csv)
# mug_csv.head()

In [ ]:
all_df = mug_csv.append(kettle_csv, ignore_index=True)
all_df = all_df.append(bottle_csv, ignore_index=True)
all_df = all_df.append(bakingtray_csv, ignore_index=True)
all_df = all_df.append(hammer_csv, ignore_index=True)
all_df = all_df.append(pan_csv, ignore_index=True)
all_df = all_df.append(wrench_csv, ignore_index=True)

print(len(all_df))
all_df.head()

In [ ]:
import string # for preprocess_text()
import re
# NLTK library to remove the stopwords
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

'''
Want to try to method
1) change the digit to word
2) split the number into single digit
'''
num_to_word = {'1' : ' one ',
               '2' : ' two ',
               '3' : ' three ',
               '4' : ' four ',
               '5' : ' five ',
               '6' : ' six ',
               '7' : ' eight ',
               '8' : ' eight ',
               '9' : ' nine ',
               '0' : ' zero '}

regex = '([0-9]+\.?[0-9]+)([a-zA-Z]+)'
r = re.compile(regex)

# unit conversion
convert_vol = {'oz'     : 29.5735,
               'cl'     : 10.0,
               'gallon' : 4546.09,
               'L'      : 1000.0}
char_set = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"

class InputPreprocess():
  @staticmethod
  def quantize(val, to_values):
      """Quantize a value with regards to a set of allowed values.
      
      Examples:
          quantize(49.513, [0, 45, 90]) -> 45
          quantize(43, [0, 10, 20, 30]) -> 30
      
      Note: function doesn't assume to_values to be sorted and
      iterates over all values (i.e. is rather slow).
      
      Args:
          val        The value to quantize
          to_values  The allowed values
      Returns:
          Closest value among allowed values.
      """
      best_match = None
      best_match_diff = None
      for other_val in to_values:
          diff = abs(other_val - val)
          if best_match is None or diff <= best_match_diff:
              best_match = other_val
              best_match_diff = diff
      return best_match

  # Preprocess weights --> e.g. 990g -> 990 <int>
  @staticmethod
  def preprocess_weights(df):
    tmp = df.copy()
    weights = tmp['Weight'].to_list()
    # Want to extract rows with 'g' only
    row_g = []
    for i in range(len(weights)):
      curr_weight = weights[i]
      if curr_weight.endswith('kg'):
        curr_weight = curr_weight[0:-2]
        curr_weight = float(curr_weight) * 1000
        # row_g.append(i)
      elif curr_weight.endswith('g'):
        curr_weight = curr_weight[0:-1]
        curr_weight = float(curr_weight)
        row_g.append(i)
      elif curr_weight.endswith('pounds'):
        curr_weight = curr_weight[0:-6]
        curr_weight = float(curr_weight) * 453.592
      elif curr_weight.endswith('oz'):
        curr_weight = curr_weight[0:-2]
        curr_weight = float(curr_weight) * 28.3495
      elif curr_weight == 'other':
        curr_weight = 0
      if curr_weight > 5000:
        curr_weight = 0
      weights[i] = int(curr_weight)
    tmp['Weight'] = weights
    return tmp, row_g
  
  @staticmethod
  def preprocess_volume(df):
    tmp = df.copy()
    volume = tmp['Volume'].to_list()
    for i in range(len(volume)):
      curr_volume = volume[i].split(';')[0] # now only consider to first element in the labels

      if curr_volume.endswith('ml'):
        curr_volume = float(curr_volume[0:-2])
      elif curr_volume.endswith('oz'):
        curr_volume = float(curr_volume[0:-2]) * convert_vol['oz']
      elif curr_volume.endswith('g'):
        curr_volume = float(curr_volume[0:-1])
      elif curr_volume.endswith('cl'):
        curr_volume = float(curr_volume[0:-2]) * convert_vol['cl']
      elif curr_volume.endswith('L'):
        curr_volume = float(curr_volume[0:-1]) * convert_vol['L']
      elif curr_volume.endswith('gallon'):
        curr_volume = float(curr_volume[0:-6]) * convert_vol['gallon']
      elif curr_volume == 'other':
        curr_volume = 0
      volume[i] = int(curr_volume)
      # if volume[i] > 5000:
      #   volume[i] = 0
    tmp['Volume'] = volume
    return tmp

  @staticmethod
  def convert_num2words(word_list):
    tmp_word_list = []
    for word in word_list:
      if word[0].isdigit() and word[-1].isdigit():
        try:
          num_in_word = num2words(float(word))
          tmp_word_list.append(num_in_word)
        except ValueError as e:
          tmp_word_list.append(word)
      else:
        tmp_word_list.append(word)
    return tmp_word_list
  @staticmethod
  def separate_num_unit(word_list):
    tmp_word_list = []
    for word in word_list:
      if word == '':
        continue
      m = r.match(word)
      if m is not None:
        tmp_word_list.extend(list(m.groups()))
      else:
        tmp_word_list.append(word)
    return tmp_word_list

  @staticmethod
  def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words
  
  @staticmethod
  def remove_out_of_char(sentence):
    res = ''
    for ch in sentence:
      if ch in char_set:
        res += ch
      else:
        res += ' '
    return res
  """
  Clean the text with the following rules
  - Convert newline \n to white space
  - Convert tab \t to white space
  - Lowercase all texts
  - Covert punctuation to white space

  """
  @staticmethod
  def preprocess_text(df, column):
    tmp = df.copy()
    for i in range(len(tmp)):
      s = tmp[column][i]
      s = s.lower()
      s = s.replace('\n', ' ') 
      s = s.replace('\t', ' ')
      s = InputPreprocess.remove_out_of_char(s)
      word_list = s.split(' ')
      # handle cases like 12oz -> 12 oz (separate the value and the unit)
      tmp_word_list = InputPreprocess.separate_num_unit(word_list)

      # word_list = InputPreprocess.convert_num2words(tmp_word_list)
      s = ' '.join(tmp_word_list)
      res_string = ''
      for j in range(len(s)):
        if s[j] == '.' and j-1 > 0 and j+1 < len(s) and s[j-1].isdigit() and s[j+1].isdigit():
          res_string += s[j]
        elif s[j] in string.punctuation:
          continue
        else:
          res_string += s[j]

      # s = " ".join("".join([" " if ch in string.punctuation else ch for ch in s]).split())
      tmp.at[i, column] = res_string.split(' ')
    # Remove stopwords using NLTK library
    print('applying NLTK')
    tmp[column] = tmp[column].apply(lambda x: InputPreprocess.remove_stopwords(x))
    for i in range(len(tmp)):
      s = tmp[column][i]
      s = ' '.join(s)
      tmp.at[i, column] = s
    return tmp
  
  @staticmethod
  def preprocess_dimensions(df):
    tmp = df.copy()
    length, width, height = tmp['Length'], tmp['Width'], tmp['Height']
    res_length, res_width, res_height = [], [], []
    for i in range(len(df)):
      if length[i] == 'other':
        res_length.append(0)
      else:
        res_length.append(int(round(float(length[i]))))

      if width[i] == 'other':
        res_width.append(0)
      else:
        res_width.append(int(round(float(width[i]))))

      if height[i] == 'other':
        res_height.append(0)
      else:
        res_height.append(int(round(float(height[i]))))
    tmp['Length'], tmp['Width'], tmp['Height'] = res_length, res_width, res_height
    return tmp

  @staticmethod
  def parts_into_list(df):
    tmp = df.copy()
    for i in range(len(tmp)):
      s = tmp['Parts'][i]
      s = s.split(';')
      tmp.at[i, 'Parts'] = s
    return tmp    

In [ ]:
'''
NOTE: Change this if needed for numeric label resolution
'''
weight_vol_res = 100
size_res = 3

# Pre-process on input text
obj_df = InputPreprocess.preprocess_text(all_df, 'Input')
obj_df = InputPreprocess.preprocess_text(obj_df, 'UnstructuredText')
obj_df = InputPreprocess.preprocess_text(obj_df, 'StructuredText')

# Pre-process on weight and volume
obj_df, row_g = InputPreprocess.preprocess_weights(obj_df)
obj_df = InputPreprocess.preprocess_volume(obj_df)
weight_bins = np.arange(min(obj_df['Weight']), max(obj_df['Weight']), weight_vol_res)
volume_bins = np.arange(min(obj_df['Volume']), max(obj_df['Volume']), weight_vol_res)
obj_df['Weight'] = obj_df['Weight'].apply(lambda x : InputPreprocess.quantize(x, weight_bins))
obj_df['Volume'] = obj_df['Volume'].apply(lambda x : InputPreprocess.quantize(x, volume_bins))

# Pre-process on the size
obj_df = InputPreprocess.preprocess_dimensions(obj_df)
length_bins = np.arange(min(obj_df['Length']), max(obj_df['Length']), size_res)
width_bins = np.arange(min(obj_df['Width']), max(obj_df['Width']), size_res)
height_bins = np.arange(min(obj_df['Height']), max(obj_df['Height']), size_res)
obj_df['Length'] = obj_df['Length'].apply(lambda x : InputPreprocess.quantize(x, length_bins))
obj_df['Width'] = obj_df['Width'].apply(lambda x : InputPreprocess.quantize(x, width_bins))
obj_df['Height'] = obj_df['Height'].apply(lambda x : InputPreprocess.quantize(x, height_bins))

# Pre-process on the object parts
obj_df = InputPreprocess.parts_into_list(obj_df)
obj_df.head()

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
obj_df['Parts'] = list(mlb.fit_transform(list(obj_df['Parts'])))
obj_df.head()

In [ ]:
fig = plt.figure(figsize=(5, 3), dpi=200)
ax1 = fig.add_subplot(1, 1, 1)
ax1.hist(obj_df['Weight'], bins=50)

plt.show()

In [ ]:
text_lens = []
for t in obj_df['Input']:
  text_lens.append(len(t))
maximum_text_length = max(text_lens)
print('Maximum text length: {}'.format(maximum_text_length))

In [ ]:
material_list = list(set(obj_df["Material"]))
print(len(material_list), material_list)

colour_list = list(set(obj_df['Colour']))
print(len(colour_list), colour_list)

weight_list = list(set(obj_df['Weight']))
print(len(weight_list), sorted(weight_list))

volume_list = list(set(obj_df['Volume']))
print(len(volume_list), sorted(volume_list))

object_list = list(set(obj_df['Object']))
print(len(object_list), sorted(object_list))

length_list = list(set(obj_df['Length']))
print(len(length_list), sorted(length_list))

width_list = list(set(obj_df['Width']))
print(len(width_list), sorted(width_list))

height_list = list(set(obj_df['Height']))
print(len(height_list), sorted(height_list))

part_list = list(mlb.classes_)
print(part_list)

In [ ]:
# Create the dictionary for material and colour
material_to_idx = {x : i for i, x in enumerate(material_list)}
colour_to_idx = {x : i for i, x in enumerate(colour_list)}
weight_to_idx = {x : i for i, x in enumerate(weight_list)}
volume_to_idx = {x : i for i, x in enumerate(volume_list)}
object_to_idx = {x : i for i, x in enumerate(object_list)}
length_to_idx = {x : i for i, x in enumerate(length_list)}
width_to_idx = {x : i for i, x in enumerate(width_list)}
height_to_idx = {x : i for i, x in enumerate(height_list)}
# Now update the label to integer form
for i in range(len(obj_df)):
  obj_df.iloc[i, 3] = material_to_idx[obj_df.iloc[i, 3]]
  obj_df.iloc[i, 4] = colour_to_idx[obj_df.iloc[i, 4]]
  obj_df.iloc[i, 5] = weight_to_idx[obj_df.iloc[i, 5]]
  obj_df.iloc[i, 6] = volume_to_idx[obj_df.iloc[i, 6]]
  obj_df.iloc[i, 7] = length_to_idx[obj_df.iloc[i, 7]]
  obj_df.iloc[i, 8] = width_to_idx[obj_df.iloc[i, 8]]
  obj_df.iloc[i, 9] = height_to_idx[obj_df.iloc[i, 9]]
  obj_df.iloc[i, -1] = object_to_idx[obj_df.iloc[i, -1]]
obj_df.head()

In [ ]:
text_preprocess_df = obj_df.copy()

In [ ]:
# Print the number of output label classes
print('Material: {} \t Colour: {} \t Weight: {}'.format(len(material_to_idx), len(colour_to_idx), len(weight_to_idx)))

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(text_preprocess_df['Input'].values)

alphabet="abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
    
# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy() 
# Add 'UNK' to the vocabulary 
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

train_sequences = tk.texts_to_sequences(text_preprocess_df['Input'].values)
train_data = pad_sequences(train_sequences, maxlen=maximum_text_length, padding='pre')

# Convert to numpy array
# train_data = np.array(train_data, dtype='float32')

# For output labels
material_labels = tf.keras.utils.to_categorical(np.asarray(text_preprocess_df['Material']))
colour_labels = tf.keras.utils.to_categorical(np.asarray(text_preprocess_df['Colour']))
weight_labels = tf.keras.utils.to_categorical(np.asarray(text_preprocess_df['Weight']))
volume_labels = tf.keras.utils.to_categorical(np.asarray(text_preprocess_df['Volume']))
object_labels = tf.keras.utils.to_categorical(np.asarray(text_preprocess_df['Object']))
length_labels = tf.keras.utils.to_categorical(np.asarray(text_preprocess_df['Length']))
width_labels = tf.keras.utils.to_categorical(np.asarray(text_preprocess_df['Width']))
height_labels = tf.keras.utils.to_categorical(np.asarray(text_preprocess_df['Height']))
part_labels = np.asarray(list(obj_df['Parts']))

In [ ]:
vocab_size = len(tk.word_index)
vocab_size

In [ ]:
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size)) # zero vector to represent the PAD

for char, i in tk.word_index.items():
  onehot = np.zeros(vocab_size)
  onehot[i-1] = 1
  embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)


In [ ]:
# First row all 0 for PAD, 68 char, last row for UNK
print(embedding_weights.shape)
print(embedding_weights)

In [ ]:
# Parameter
input_size = maximum_text_length
embedding_size = 69
# filter_num, filter_size, pooling_size
conv_layers = [[256, 7, 3],
               [256, 7, 3],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, 3]]
fully_connected_layers = [1024, 1024]
num_of_material_classes = len(material_to_idx)
num_of_colour_classes = len(colour_to_idx)
num_of_weight_classes = len(weight_to_idx)
num_of_volume_classes = len(volume_to_idx)
num_of_object_classes = len(object_to_idx)
num_of_length_classes = len(length_to_idx)
num_of_width_classes = len(width_to_idx)
num_of_height_classes = len(height_to_idx)
num_of_part_classes = len(part_list)
dropout_p = 0.5
optimizer = 'adam'


In [ ]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model

# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

inputs = Input(shape=(input_size,), name='input', dtype='int64')

x = embedding_layer(inputs)
# Convolution layers
for filter_num, filter_size, pooling_size in conv_layers:
  x = Conv1D(filter_num, filter_size)(x)
  x = Activation('relu')(x)
  if pooling_size != -1:
    x = MaxPooling1D(pool_size=pooling_size)(x)

x = Flatten()(x)

for dense_size in fully_connected_layers:
  x = Dense(dense_size, activation='relu')(x)
  x = Dropout(dropout_p)(x)

material_branch = Dense(num_of_material_classes, activation='softmax', name='material_output')(x)
colour_branch = Dense(num_of_colour_classes, activation='softmax', name='colour_output')(x)
weight_branch = Dense(num_of_weight_classes, activation='softmax', name='weight_output')(x)
volume_branch = Dense(num_of_volume_classes, activation='softmax', name='volume_output')(x)
object_branch = Dense(num_of_object_classes, activation='softmax', name='object_output')(x)
length_branch = Dense(num_of_length_classes, activation='softmax', name='length_output')(x)
width_branch = Dense(num_of_width_classes, activation='softmax', name='width_output')(x)
height_branch = Dense(num_of_height_classes, activation='softmax', name='height_output')(x)
part_branch = Dense(num_of_part_classes, activation='sigmoid', name='part_output')(x)







# Build model
'''
Multiple output
'''
model = Model(inputs=inputs,  outputs=[material_branch,
                                       colour_branch,
                                       weight_branch,
                                       volume_branch,
                                       object_branch,
                                       length_branch,
                                       width_branch,
                                       height_branch,
                                       part_branch])
'''
Single output
'''

# outputs = part_branch
# model = Model(inputs=inputs, outputs=outputs)

'''
Change this for multiple or single output
'''
losses = {
    "material_output": "categorical_crossentropy",
    "colour_output": "categorical_crossentropy",
    "weight_output": "categorical_crossentropy",
    "volume_output": "categorical_crossentropy",
    "object_output": "categorical_crossentropy",
    "length_output": "categorical_crossentropy",
    "width_output": "categorical_crossentropy",
    "height_output": "categorical_crossentropy",
    'part_output' : 'binary_crossentropy'
}
# loss = 'binary_crossentropy'
model.compile(optimizer=optimizer, loss=losses, metrics=['accuracy'])
model.summary()

In [ ]:
'''
Plot model
'''
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
'''
Prepare data
'''
from sklearn.model_selection import train_test_split

# Model training params
# new_input_list or data1 variable
split_data = train_test_split(train_data,
                              material_labels,
                              colour_labels,
                              weight_labels,
                              volume_labels,
                              object_labels,
                              length_labels,
                              width_labels,
                              height_labels,
                              part_labels,
                              test_size=0.20, random_state=33)
(x_train, x_test,
 y_train_material, y_test_material,
 y_train_colour, y_test_colour,
 y_train_weight, y_test_weight,
 y_train_volume, y_test_volume,
 y_train_object, y_test_object,
 y_train_length, y_test_length,
 y_train_width, y_test_width,
 y_train_height, y_test_height,
 y_train_parts, y_test_parts) = split_data

In [ ]:
# Set up directory to save best model
!rm -rf best_model
!mkdir best_model

path = './best_model'
checkpoint = tf.keras.callbacks.ModelCheckpoint(path, monitor='val_accuracy', save_best_only=True, verbose=1)

In [ ]:

epochs = 100
# model.fit(x_train, y_train_parts, validation_data=(x_test, y_test_parts), batch_size=32, epochs=epochs, verbose=1)

history = model.fit(x=x_train,
                    y={"material_output": y_train_material,
                       "colour_output": y_train_colour,
                       'weight_output': y_train_weight,
                       'volume_output' : y_train_volume,
                       'object_output' : y_train_object,
                       'length_output' : y_train_length,
                       'width_output' : y_train_width,
                       'height_output' : y_train_height,
                       'part_output' : y_train_parts},
                    batch_size=32,
                    validation_data=(x_test,
                                     {"material_output": y_test_material,
                                      "colour_output": y_test_colour,
                                      'weight_output': y_test_weight,
                                      'volume_output' : y_test_volume,
                                      'object_output' : y_test_object,
                                      'length_output' : y_test_length,
                                      'width_output' : y_test_width,
                                      'height_output' : y_test_height,
                                      'part_output' : y_test_parts}),
                    # callbacks=[checkpoint],
                    epochs=epochs,
                    verbose=1)


In [ ]:
'''
 Load weight if use best model only 
'''
# model.load_weights(path)
pred = model.predict(x_test)

In [ ]:
idx_to_weight = {v: k for k, v in weight_to_idx.items()}
idx_to_volume = {v: k for k, v in volume_to_idx.items()}
idx_to_length = {v: k for k, v in length_to_idx.items()}
idx_to_width = {v: k for k, v in width_to_idx.items()}
idx_to_height = {v: k for k, v in height_to_idx.items()}

In [ ]:
# Prepare ground truth (from onehot back to actual measurement)
def get_numeric_true_metric(onehot, idx_dict):
  num_arr = []
  for i in onehot:
    num_arr.append(int(idx_dict[np.argmax(i)]))
  return num_arr
true_weight = get_numeric_true_metric(y_test_weight, idx_to_weight)
true_volume = get_numeric_true_metric(y_test_volume, idx_to_volume)
true_length = get_numeric_true_metric(y_test_length, idx_to_length)
true_width = get_numeric_true_metric(y_test_width, idx_to_width)
true_height = get_numeric_true_metric(y_test_height, idx_to_height)

In [ ]:
# Now look at the predicted values
# Material -> Colour -> Weight -> Volume -> Object -> Length -> Width -> Height
# Index of the property relating to the models
# prop_idx = [2, 3, 5, 6, 7]
'''
NOTE: this needs to be changed if the model is single output
for example
pred_weight, pred_weight_val = pred, []
pred_volume, pred_volume_val = pred, []
pred_length, pred_length_val = pred, []
pred_width, pred_width_val = pred, []
pred_height, pred_height_val = pred, []
'''
pred_weight, pred_weight_val = pred[2], []
pred_volume, pred_volume_val = pred[3], []
pred_length, pred_length_val = pred[5], []
pred_width, pred_width_val = pred[6], []
pred_height, pred_height_val = pred[7], []

def convert_to_number(pred, val, idx_dict):
  for i in range(len(pred)):
    val.append(int(idx_dict[np.argmax(pred[i])]))
convert_to_number(pred_weight, pred_weight_val, idx_to_weight)
convert_to_number(pred_volume, pred_volume_val, idx_to_volume)
convert_to_number(pred_length, pred_length_val, idx_to_length)
convert_to_number(pred_width, pred_width_val, idx_to_width)
convert_to_number(pred_height, pred_height_val, idx_to_height)


In [ ]:
def get_abs_error(true, pred):
  abs_arr = []
  for i in range(len(true)):
    abs_arr.append(abs(true[i] - pred[i]))
  return abs_arr

weight_abs_error = np.asarray(get_abs_error(true_weight, pred_weight_val))
volume_abs_error = np.asarray(get_abs_error(true_volume, pred_volume_val))
length_abs_error = np.asarray(get_abs_error(true_length, pred_length_val))
width_abs_error = np.asarray(get_abs_error(true_width, pred_width_val))
height_abs_error = np.asarray(get_abs_error(true_height, pred_height_val))

print(f'Mean absolute error of weight: {np.mean(weight_abs_error)}g')
print(f'Mean absolute error of volume: {np.mean(volume_abs_error)}ml')
print(f'Mean absolute error of length: {np.mean(length_abs_error)}cm')
print(f'Mean absolute error of width : {np.mean(width_abs_error)}cm')
print(f'Mean absolute error of height: {np.mean(height_abs_error)}cm')

In [ ]:

!mkdir ./experiment4_cnn/

In [ ]:
plt.style.use("ggplot")


In [ ]:
def plot_abs_error(abs_error, property):
  abs_error = abs_error[abs_error != 0]
  weights = np.ones_like(abs_error) / len(abs_error)
  fig = plt.figure(figsize=(10, 4), dpi=200)
  ax1 = fig.add_subplot(1, 1, 1)
  ax1.hist(abs_error, weights=weights, bins=200)
  ax1.set_xlabel('Absolute Error')
  ax1.set_ylabel('Normalized Frequency')
  ax1.set_title(f'Absolute error on {property} prediction')
  plt.savefig(f'./experiment4_cnn/{property}.png')

plot_abs_error(weight_abs_error, 'weight')
plot_abs_error(volume_abs_error, 'volume')
plot_abs_error(length_abs_error, 'length')
plot_abs_error(width_abs_error, 'width')
plot_abs_error(height_abs_error, 'height')


In [ ]:
!zip -r download4.zip ./experiment4_cnn